In [2]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functools

%matplotlib inline


In [5]:
def statistics_vs_probability():
    return """
    data ---stats--> causes
    data <--prob---- causes
    """
print(statistics_vs_probability())


    data ---stats--> causes
    data <--prob---- causes
    


# Bayes Rule
 - Reverend Thomas Bayes
 - To infers existence of God
 - Informed Artificial Intelligence


In [8]:
def cancer_test():
    """
    :returns: P(C|P);
    Prob you have cancer given the test is positive
    Solution: https://www.udacity.com/course/viewer#!/c-st101/l-48703346/e-48759018/m-48480554
    
    """
    #returning this suggests your chances of having cancer
    #Haven't changed regardless of the test returning positive
    p_c = 0.01 
    
    #Returning this is an answer to a different question;
    #this is the probability of positive test result
    #This question asks about the probability of cancer
    p_p_g_c = 0.9
    
    p_c_g_p = 0.08
    
    #Since sensitivity (i.e. the amount of confidence we can put)
    #That a positive test means a positive cancer?
    #Is high; we expect it increases our chances from the generic population
    assert p_c_g_p > p_c
    
    #And no relation to p_p_g_c; different question
    
    return p_c_g_p
    

In [3]:
class Event(object):
    
    GIVEN = ' given '
    NOT = 'not'
    AND = ' and '
    NEGATIVE = 'negative'
    POSITIVE = 'positive'
    
    #def __new__(cls, name, prob, prob_universeprob_population={}):
    #    return super().__new__(cls, name, prob, prob_population)
    # Was gonna check first the population, but...
    
    def __init__(self, name):#, *args, **kwargs):
        self.__name = str(name).strip()
        
    def __neg__(self):
        return self.__invert__()
    
    def __invert__(self):
        """
        The idea of a binary event
        :returns: the 'inverse' of this event; this event not-happening
        """
        # A conditional event; just negate the posterior
        if self.given():
            return (~self.posterior()) | self.given()
        
        # A joint event; negate both and re-join
        if all(self.joint()):
            return ~self.joint()[0]  & ~self.joint()[1]
        
        # A negative single event; make positive
        if self.__name.startswith(self.NOT):
            return Event(self.__name.lstrip(self.NOT).strip('()'))
            
        # A positive single event; make negative
        else:
            #if self.POSITIVE in self.__name:
            #    not_name = self.__name.replace(self.POSITIVE, self.NEGATIVE)
            #elif self.NEGATIVE in self.__name:
            #    not_name = self.__name.replace(self.NEGATIVE, self.POSITIVE)
            #else:
            return Event('{0}({1})'.format(self.NOT, self.__name))
        
    def __or__(self, event_2):
        """Notice this would *not* support
        transitive property as "or" normally should
        Though I guess "truthied" things aren't actually transitive
        (A__or__B == B__or__A)
        """
        return self.given(event_2)
    
    def __and__(self, event_2):
        """Notice this would *not* support
        transitive property as "or" normally should
        Though I guess "truthied" things aren't actually transitive
        (A__or__B == B__or__A)
        """
        return self.joint(event_2)
    
    def joint(self, event_2=None):
        if event_2 is None:
            if self.AND in self.__name:
                return [Event(_) for _ in self.__name.split(self.AND)]
            else:
                return [None, None]
        else:
            and_name = '{0}{1}{2}'.format(self, self.AND, event_2)
            return Event(and_name)

    def posterior(self):
        """
        If this is a conditional,
        P(Cancer|Pos)
        :returns: the name of the posterior event, that the patient has Cancer
        """
        #GET method
        if self.GIVEN in self.__name:
            #assert self.GIVEN in self.__name, 'No second event'
            return Event(self.__name.split(self.GIVEN)[0])
        #No SET method because there is not left-to-right syntax
        #Whereas the "given" method creates a new conditional
            
            
   
    def given(self, event_2=None):
        """
        If this is a conditional, event
        P(Cancer|Pos)
        :params event_2: the second event IF you want to create a new given event
        :returns: the name of the original event, that test is Positive, if no event_2 param passed and this is a conditional event
        Else a new conditional event created by combining self and event_2
        """
        #GET method
        if event_2 is None:
            
            if self.GIVEN in self.__name:
                #assert self.GIVEN in self.__name, 'No first event'
                return Event(self.__name.split(self.GIVEN)[1])
        #SET method
        else:
            given_name = '{0}{1}{2}'.format(self, self.GIVEN, event_2)
            #given_prob = prob
            return Event(given_name) #, given_prob, self.__probs)
   
    def __nickname__(self):
        """
        Would want nicknames for like 'gay' could by 'G'
        And other  conventions (saved as constants) like 'N' for 'not'
        
        'G' for given; or the symbol '|'
        'A' for and; or the symbol or acceptable '&'
        """
        return (self.__name
                .replace(' given ', '|')
                .replace(' and ', ',')
                .upper())
        
    def __repr__(self):
        return 'Event("{}")'.format(self.__name)

    def __str__(self):
        return self.__name

    def __hash__(self):
        return hash(tuple(sorted(self.__name.split(self.AND))))

    def __eq__(self, other):
        return hash(self) == hash(other)

In [4]:
class Probability(dict):
        
    def __call__(self, event, *args, **kwargs):
        return self.__getitem__(event)
        
    def __getitem__(self, key):
        """
        Not a sequence type;
        Not implementing negative integers or accepting ranges
        """
        
        return super().__getitem__(key)
    
    def __set__(self, event, prob):
       
        assert event is not None, 'Event must be not None'
            
        prob = float(prob)
        event = Event(event)
        
        assert 0 <= prob <= 1, 'Dont know probability of {0} ' \
        '{1} outside of limits 0 and 1'.format(event, prob) 
        
        super().__setitem__(event, prob)
        
        #Create Joint from Conditional and Single matching Original Event
        #=================
        #Scenario 1; event = P(A|B) and already have P(B)
        #Can create P(A&B)
        
        try:
            conditional_event = event
            conditional_prob = prob
            original_event = event.given()
            original_prob = self[original_event]
            
            joint_event = conditional_event.posterior() & conditional_event.given()
            joint_prob = conditional_prob * original_prob
            #print('joint')
            super().__setitem__(joint_event, joint_prob)
            
        except KeyError as e:
            msg = 'dont have P({})'.format(original_event) if original_event is not None else 'no original event'
            pass#print('No joint, {}'.format(msg))
            
        #Create Joint from Single and any matching Conditional
        #================
        #Scenario 2; event = P(B) and already have P(N|B); 
        #    notice how "N" could be anything! Could be more than one; for-loop it
        #Can create P(N&B)
        try:
            original_prob = prob
            original_event = event
            conditional_event = [_ for _ in self.events() if _.given()==original_event][0]
  
            for conditional_event in conditional_events:
                conditional_prob = self[conditional_event]
                
                joint_event = conditional_event.posterior() & conditional_event.given()
                joint_prob = conditional_prob * original_prob
                
                #print('joint')
                super().__setitem__(joint_event, joint_prob)
            
        except Exception as e:
            pass#print('No joint, dont have events with original {}'.format(original_event))#pass#print('No scenario 2')
            
        
        #Create Single from Joint and other Single in Joint
        #====================
        #Scenario one; event = P(A&B) and we have P(A)
        #Can create P(B) 
        #or we have P(B) and we can create P(A)
        e_1, e_2 = event.joint() 
        joint_event = event
        joint_prob = prob
        for original_event, second_event in ((e_1, e_2), (e_2, e_1)):
            try:
                original_prob = self[original_event]
                
                conditional_event = second_event | original_event
                conditional_event_prob = joint_prob / original_prob
                
                #print('conditional')
                super().__setitem__(conditional_event, conditional_event_prob)
            except Exception as e:
                pass#print('No conditional')#pass
            

    def events(self):
        return self.keys()
    
    def __setitem__(self, event, prob):
        for event, prob in ((event, prob), (~event, 1-prob)):
            print(event, prob)
            self.__set__(event, prob)
        print('--')

In [5]:

P = Probability()
C = Event('cancer')
Pos = Event('positive') 
Neg = ~Pos

P[C] = 0.01
P[Pos | C] = 0.9
P[Neg | ~C] = 0.9

print(P(~C) * P(Pos | ~C))
print(P[~C & Pos])


cancer 0.01
not(cancer) 0.99
--
positive given cancer 0.9
not(positive) given cancer 0.09999999999999998
--
not(positive) given not(cancer) 0.9
positive given not(cancer) 0.09999999999999998
--
0.09899999999999998
0.09899999999999998


In [554]:
(cancer | pos).joint()

[None, None]

In [555]:
(cancer & pos).joint()

[Event("cancer"), Event("test positive")]

In [556]:
import pprint
pprint.pprint(list(P.items()))

[(Event("positive given cancer"), 0.09999999999999998),
 (Event("not cancer"), 0.99),
 (Event("not positive and cancer"), 0.991),
 (Event("cancer"), 0.01),
 (Event("not positive and not cancer"), 0.891),
 (Event("not positive given not cancer"), 0.9),
 (Event("not positive given cancer"), 0.09999999999999998),
 (Event("positive and cancer"), 0.10899999999999999)]


TypeError: __init__() takes 2 positional arguments but 3 were given

In [264]:
P.all().keys()

dict_keys([<__main__.Event object at 0x0000000005411E48>])

In [236]:
'Event("cancer", 0.5)' == 'Event("cancer", 0.5)'

True